In [31]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, Embedding, Dropout, Activation, Flatten, Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model, Sequential
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.layers import LSTM
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from tqdm import tqdm
from nltk.tokenize import RegexpTokenizer 

In [7]:
cod_train = pd.read_csv("Data Skripsi Anne NoTgl.csv", encoding="latin-1")

In [8]:
cod_train2, test= train_test_split(cod_train, random_state=0, test_size=0.2)
train, val =  train_test_split(cod_train2, random_state = 0,test_size=0.2)

In [9]:
train['Isi Berita'].head()

752     JAKARTA - Sebanyak 18 pegawai Komisi Pemberant...
709     JAKARTA - Pemberlakuan pembatasan kegiatan mas...
2958    JAKARTA- Kasus positifCovid-19kembali bertamba...
3319    JAKARTA   Pandemi Covid-19 yang melanda Tanah...
844     JAKARTA - Menteri Koordinator Bidang Kemaritim...
Name: Isi Berita, dtype: object

In [10]:
max_features = 100000 # max num words
maxlen = 250 
embedding_size = 200

# create the tokenizer with the maximum number of words to keep, 
# based on word frequency. 
# Only the most common num_words-1 words will be kept.
tokenizer = Tokenizer(num_words=max_features, oov_token = True)

train['Isi Berita']=train['Isi Berita'].astype(str)
test['Isi Berita']=test['Isi Berita'].astype(str)
val['Isi Berita']=val['Isi Berita'].astype(str)

# fit the tokenizer on the headlines
tokenizer.fit_on_texts(list(train['Isi Berita']))

# Transforms each text in texts to a sequence of integers.
train_X = tokenizer.texts_to_sequences(train['Isi Berita'])
test_X = tokenizer.texts_to_sequences(test['Isi Berita'])
val_X = tokenizer.texts_to_sequences(val['Isi Berita'])

# transforms a list of num_samples sequences (lists of integers)
# into a 2D Numpy array of shape (num_samples, num_timesteps).
train_X = pad_sequences(train_X, maxlen = maxlen)
test_X = pad_sequences(test_X, maxlen = maxlen)
val_X = pad_sequences(val_X, maxlen = maxlen)

train_y = train['Label']
test_y = test['Label']
val_y = val['Label']

/home/candra/miniconda3/envs/env-wrap/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/home/candra/miniconda3/envs/env-wrap/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/home/candra/miniconda3/envs/env-wrap/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is 

In [24]:
def preprocess_dataset(X_train, X_test):
    raw_docs_train = X_train
    raw_docs_test = X_test
    tokenizer = RegexpTokenizer(r'\w+')
    max_seq_len = 20
    num_classes = 1

    print("pre-processing train data...")
    processed_docs_train = []
    for doc in tqdm(raw_docs_train):
        tokens = tokenizer.tokenize(doc)
        filtered = [word for word in tokens]
        processed_docs_train.append(" ".join(filtered))

    processed_docs_test = []
    for doc in tqdm(raw_docs_test):
        tokens = tokenizer.tokenize(doc)
        filtered = [word for word in tokens]
        processed_docs_test.append(" ".join(filtered))
        
    return processed_docs_train, processed_docs_test

In [26]:
def create_word_index_dict(all_processed_data):
    tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower=True, char_level=False)
    tokenizer.fit_on_texts(all_processed_data)
    word_index = tokenizer.word_index
    print("dictionary size: ", len(word_index))
    
    return tokenizer

In [29]:
## download cc.id vector https://www.kaggle.com/khotijahs1/ccid300vecgz
def load_indonesian_word_embeddings():
    import codecs
    indonesian_embeddings_index = {}
    fasttext_indo = codecs.open('cc.id.300.vec', encoding='utf-8')
    for line in tqdm(fasttext_indo):
        values = line.rstrip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        indonesian_embeddings_index[word] = coefs

    fasttext_indo.close()
    print('found %s word vectors' % len(indonesian_embeddings_index))
    
    return indonesian_embeddings_index

In [ ]:
def plot_embedding_matrix(word_index, embeddings_index):
    embed_dim = 300
    words_not_found = []
    MAX_NB_WORDS = 30000
    
    print('preparing embedding matrix...')

    nb_words = min(MAX_NB_WORDS, len(word_index)+1)
    
    embedding_matrix = np.zeros((nb_words, embed_dim))

    for word, i in word_index.items():
        if i >= nb_words:
            continue
        embedding_vector = embeddings_index.get(word)
        if (embedding_vector is not None) and len(embedding_vector) > 0:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector

        else:
            words_not_found.append(word)
    print('number of null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))
    print("sample words not found: ", np.random.choice(words_not_found, 10))
    
    return embedding_matrix, nb_words

In [ ]:
# EMBEDDING_FILE = 'vocab.txt'

# def get_coefs(word,*arr): 
#     return word, np.asarray(arr, dtype='float32')

# embeddings_index = {}
# with open(EMBEDDING_FILE, encoding="utf8") as f:
#     for line in f:
#         word, coefs = get_coefs(*line.split(" "))
#         embeddings_index[word] = coefs
            
# all_embs = np.stack(embeddings_index.values())
# emb_mean,emb_std = all_embs.mean(), all_embs.std()
# embed_size = all_embs.shape[1]

# word_index = tokenizer.word_index
# nb_words = min(max_features, len(word_index))

# embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embedding_size))
# for word, i in word_index.items():
#     if i >= max_features: 
#         continue
#     embedding_vector = embeddings_index.get(word)
#     if embedding_vector is not None: 
#         embedding_matrix[i] = embedding_vector


embedding_index = load_indonesian_word_embeddings()
train_data_processed, test_data_processed = preprocess_dataset(train_X, test_X)
tokenizer = create_word_index_dict(train_data_processed+test_data_processed)
word_index = tokenizer.word_index
embedding_matrix, nb_words = plot_embedding_matrix(word_index, embedding_index)


2000001it [07:28, 4455.75it/s] 

In [ ]:
model = Sequential()
model.add(Embedding(max_features, embedding_size, weights = [embedding_matrix]))
model.add(Bidirectional(LSTM(128, return_sequences = True)))
model.add(GlobalMaxPool1D())
model.add(Dense(40, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',keras.metrics.Precision(), keras.metrics.Recall(), keras.metrics.TruePositives()])

In [ ]:
# Save the model after every epoch.
saveBestModel = keras.callbacks.ModelCheckpoint('best_model.hdf5', monitor='val_acc', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
# Stop training when a monitored quantity has stopped improving.
earlyStopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')

In [ ]:
batch_size = 100
epochs = 25
model.fit(train_X, train_y, batch_size=batch_size, epochs=epochs, validation_data=(val_X, val_y), callbacks=[saveBestModel, earlyStopping])

In [ ]:
model.metrics_names

In [ ]:
loss, accuracy, precision, recall, true_positives = model.evaluate(test_X, test_y, batch_size=batch_size)

In [ ]:
mult_pr=precision*recall
sum_pr=precision+recall
div=mult_pr/sum_pr
f1_score=2*div

In [ ]:
print('Loss:',loss)
print('Accuracy:',accuracy)
print('Precision:',precision)
print('Recall:',recall)
print('f1 score:',f1_score)
print('True positives:',true_positives)

In [ ]:
pred_y = model.predict_classes(test_X, batch_size=batch_size)

In [ ]:
confusion_matrix(test_y, pred_y)

In [ ]:
def getFP_FN_TP_lists(test_X, test_y, pred_y):
    FP_text = []
    FP_index = []
    FN_text = []
    FN_index = []
    TP_text = []
    TP_index = []
    for i in range(len(test_y)):
        if(pred_y[i]==1 and test_y[test_y.index[i]]==0):
            FP_text.append(test['text'][test_y.index[i]])
            FP_index.append(test_y.index[i])
        elif(pred_y[i]==0 and test_y[test_y.index[i]]==1):
            FN_text.append(test['text'][test_y.index[i]])
            FN_index.append(test_y.index[i])
        elif(pred_y[i]==1 and test_y[test_y.index[i]]==1):
            TP_text.append(test['text'][test_y.index[i]])
            TP_index.append(test_y.index[i])        
            
    return FP_text,FP_index,FN_text,FN_index,TP_text,TP_index

def getFP_FN_TP(test_X, test_y, pred_y):
    FP_text,FP_index,FN_text,FN_index,TP_text,TP_index = getFP_FN_TP_lists(test_X, test_y, pred_y)
    d_FP = {'FP_text':FP_text,'FP_index':FP_index}
    df_FP = pd.DataFrame(d_FP)
    d_FN = {'FN_text':FN_text,'FN_index':FN_index}
    df_FN = pd.DataFrame(d_FN)
    d_TP =  {'TP_text':TP_text,'TP_index':TP_index}
    df_TP = pd.DataFrame(d_TP)
    
    return df_FP,df_FN,df_TP

df_FP,df_FN, df_TP = getFP_FN_TP(test_X, test_y, pred_y)
df_FP.to_csv('FP_BiLSTMG_CB.csv', index=True)
df_FN.to_csv('FN_BiLSTM_CB.csv', index=True)
df_TP.to_csv('TP_BiLSTM_CB.csv', index=True)